```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de Datos no Estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago
```

# Resolviendo un problema de clasificación utilizando word2vec

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [4]:
from TextNormalizer import TweetTextNormalizer

AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

In [ ]:
normalizer = TweetTextNormalizer(text_to_sequence=True, lemmatize=False)

In [2]:
from Word2VecVectorizer import Word2VecVectorizer

ModuleNotFoundError: No module named 'sklearn'

In [6]:
w2v = Word2VecVectorizer(model_path='Models/Word2Vec/model.bin', sequence_to_idx=True)
embedding_weights = w2v.get_weights()

In [7]:
from PadSequenceTransformer import PadSequenceTransformer

In [8]:
seq2seq = PadSequenceTransformer(max_len=40)

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D

In [10]:
def build_model(sequence_len, vocab_size, emdedding_size, embedding_weights):
    sequences = Input(shape=(sequence_len,))
    embeddings = Embedding(vocab_size, emdedding_size,
                           weights=[embedding_weights],
                           trainable=False,
                           mask_zero=True)(sequences)
    embedded_sequences = SpatialDropout1D(0.2)(embeddings)
    lstm = LSTM(emdedding_size)(embedded_sequences)
    topics = Dense(7, activation='softmax')(lstm)
    
    model = Model(inputs=[sequences], outputs=topics)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.AUC()])
    return model

In [11]:
estimator = keras.wrappers.scikit_learn.KerasClassifier(
               build_fn=build_model, 
               epochs=100,
               sequence_len=40,
               vocab_size=w2v.vocab_size,
               emdedding_size=w2v.emdedding_size,
               embedding_weights=embedding_weights
)

In [12]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', w2v),
                           ('padder', seq2seq),
                           ('estimator', estimator)])

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], test_size=0.33, stratify=tweets['SECTOR'])

In [14]:
model = pipeline.fit(X=X_train, y=y_train)

AttributeError: 'Embedding' object has no attribute 'shape'

In [ ]:
train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence[:-1]):
        train_x[i, t] = w2v.word2idx(word)
        train_y[i] = w2v.word2idx(sentence[-1])